In [1]:
import torch
from torch.utils.data import Dataset, DataLoader, random_split, TensorDataset
import torch.nn as nn
import torch.optim as optim
from sklearn.preprocessing import StandardScaler
import pandas as pd

In [2]:
df = pd.read_csv('dataset.csv')

In [3]:
display(df.head())  

,Surname,CreditScore,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,...,France,Germany,Spain,Female,Male,Mem__no__Products,Cred_Bal_Sal,Bal_sal,Tenure_Age,Age_Tenure_product
0,2023,668.0,33.0,3.0,0.00,2.0,1,0,181449.97,0,...,1,0,0,0,1,0.0,0.000000,0.000000,0.090909,99.0
1,2024,627.0,33.0,1.0,0.00,2.0,1,1,49503.50,0,...,1,0,0,0,1,2.0,0.000000,0.000000,0.030303,33.0
2,1236,678.0,40.0,10.0,0.00,2.0,1,0,184866.69,0,...,1,0,0,0,1,0.0,0.000000,0.000000,0.250000,400.0
3,1362,581.0,34.0,2.0,148882.54,1.0,1,1,84560.88,0,...,1,0,0,0,1,1.0,1022.940581,1.760655,0.058824,68.0
4,491,716.0,33.0,5.0,0.00,2.0,1,1,15068.83,0,...,0,0,1,0,1,2.0,0.000000,0.000000,0.151515,165.0


In [4]:
variables = ["CreditScore", "Age", "Tenure", "Balance", "NumOfProducts", "HasCrCard", "IsActiveMember", "EstimatedSalary", "Male"]
X = df[variables]
display(X.head())

,CreditScore,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Male
0,668.0,33.0,3.0,0.00,2.0,1,0,181449.97,1
1,627.0,33.0,1.0,0.00,2.0,1,1,49503.50,1
2,678.0,40.0,10.0,0.00,2.0,1,0,184866.69,1
3,581.0,34.0,2.0,148882.54,1.0,1,1,84560.88,1
4,716.0,33.0,5.0,0.00,2.0,1,1,15068.83,1


In [5]:
y = df["Exited"]
display(y.head())

0    0
1    0
2    0
3    0
4    0
Name: Exited, dtype: int64

In [6]:
#X = (X - X.mean()) / X.std()
scaler = StandardScaler()
X = scaler.fit_transform(X)

X_tensor = torch.tensor(X, dtype=torch.float32)
y_tensor = torch.tensor(y.values, dtype=torch.float32)

bank_churn_dataset = TensorDataset(X_tensor, y_tensor)

In [7]:
# Split the dataset into train, dev, and test sets
train_size = int(0.8 * len(bank_churn_dataset))  # 80% for training
dev_size = int(0.1 * len(bank_churn_dataset))  # 10% for validation (dev)
test_size = len(bank_churn_dataset) - train_size - dev_size  # Remaining 10% for testing

train_data, dev_data, test_data = random_split(bank_churn_dataset, [train_size, dev_size, test_size])

In [8]:
batch_size = 64

# Create DataLoader for train, dev, and test sets
train_loader = DataLoader(train_data, batch_size=batch_size, shuffle=True)
dev_loader = DataLoader(dev_data, batch_size=batch_size, shuffle=False)
test_loader = DataLoader(test_data, batch_size=batch_size, shuffle=False)

In [9]:
class simpleNN(nn.Module):
    def __init__(self):
        super(simpleNN, self).__init__()
        self.fc1 = nn.Linear(9, 64)  # Input layer
        self.fc2 = nn.Linear(64, 32)  # Hidden layer
        self.fc3 = nn.Linear(32, 1)  # Output layer
        self.dropout = nn.Dropout(0.5)
        self.relu = nn.ReLU()
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        x = self.relu(self.fc1(x))
        x = self.dropout(x)
        x = self.relu(self.fc2(x))
        x = self.dropout(x)
        x = self.sigmoid(self.fc3(x))
        return x

In [10]:
# Instantiate the model, loss function, and optimizer
model = simpleNN()
criterion = nn.BCELoss()
optimizer = optim.SGD(model.parameters(), lr=0.01)

In [11]:
# Training loop
epochs = 21

for epoch in range(epochs):
    model.train()  # Set the model to training mode
    running_loss = 0

    for features, labels in train_loader:
        # Zero the parameter gradients
        optimizer.zero_grad()

        # Forward pass
        outputs = model(features)
        loss = criterion(outputs.squeeze(), labels)

        # Backward pass and optimize
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    # Print loss for the epoch
    if epoch % 10 == 0:
        print(f'Epoch [{epoch+1}/{epochs}], Loss: {running_loss/len(train_loader):.4f}')


Epoch [1/21], Loss: 0.4788
Epoch [11/21], Loss: 0.3745
Epoch [21/21], Loss: 0.3624


In [12]:
import torch.nn.functional as F

def evaluate_model(loader):
    model.eval()  # Set the model to evaluation mode
    total, correct = 0, 0
    with torch.no_grad():  # Disable gradient calculation
        for features, labels in loader:
            outputs = model(features)
            predicted = torch.argmax(outputs, dim=1)            
            total += labels.size(0)
            correct += (predicted == labels.squeeze()).sum().item()
    return correct / total

accuracy = evaluate_model(train_loader)
print(f'Train Accuracy: {accuracy * 100:.2f}%')
# Evaluate on dev set
accuracy = evaluate_model(dev_loader)
print(f'Validation Accuracy: {accuracy * 100:.2f}%')

Train Accuracy: 78.89%
Validation Accuracy: 79.01%


In [13]:
#
